<a href="https://colab.research.google.com/github/vedvkandge2000/Deep-Learning-Project/blob/master/MNIST_Dataset_using_neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.autograd as Variable

In [ ]:
input_size = 784      # Number of neurons in input layer
hidden_size = 400     # Number of neurons in hidden layer
out_size = 10         # Number of neurons in output_layer
epochs = 10           # How many times we pass entire dataset to our model
batch_size = 100      # Input Size of the data during one iteration
learning_rate = 0.001 # How fast are learing is

In [ ]:
train_dataset = datasets.MNIST(root="/content/data",
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)
test_dataset = datasets.MNIST(root="/content/data",
                              train=False,
                              transform=transforms.ToTensor())

In [ ]:
# To divied the data into different batches we uses dataloader. shuffle the data to make it independent of order
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)

In [ ]:
# Defining network class
class Net(nn.Module):
  def __init__(self,input_size,hidden_size,out_size):
    super(Net,self).__init__()
    self.fc1 = nn.Linear(input_size,hidden_size)   # First layer
    self.relu = nn.ReLU()                          # First layer activation function
    self.fc2 = nn.Linear(hidden_size,hidden_size)  # Second layer 
    self.fc3 = nn.Linear(hidden_size,out_size)     # Final layer
    # we can apply batch normalisation layer.

  def forward(self,x):
    out = self.fc1(x)
    out = self.relu(out)
    out = self.fc2(out)
    out = self.relu(out)
    out = self.fc3(out)
    return out

In [ ]:
# Creating the object
net = Net(input_size,hidden_size,out_size)
CUDA = torch.cuda.is_available()
if CUDA:
  net = net.cuda()
# Loss function. We are using cross entropy loss which comes along with softmax because this generally used for image classification.
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

In [ ]:
# Train Model
correct_train = 0
total_train = 0
for epoch in range(epochs):
  for i, (images,labels) in enumerate(train_loader):
    # Flatten the image of form (batch,1,28,28) --> (100,1,28,28) to reshape it to (100,784)
    images = images.view(-1, 28*28)
    if CUDA:
      images = images.cuda()
      labels = labels.cuda() 
    # Clear the params_grad in param = param - lr * praram_grad , so it won't get accumilated
    optimizer.zero_grad()
    outputs = net(images)  # forward prop
    _, predicted = torch.max(outputs.data,1)  # Returns the second argu of torch.max i.e index of max prob. (total of 10)

    total_train += labels.size(0)
    if CUDA:
      correct_train += (predicted.cpu() == labels.cpu()).sum()
    else:
      correct_train += (predicted == labels).sum()
    
    loss = criterion(outputs,labels)   # Difference between actual and predictd label (loss function)
    loss.backward()                    # Backpropagation
    optimizer.step()                   # Updates the weight

    if (i+1) % 100 == 0:
      print("Epoch [{}/{}], Iteration [{}/{}], Training Loss: {}, training accuracy: {}%".format
            (epoch+1,epochs,i+1, len(train_dataset)//batch_size, loss.data,(100*correct_train/total_train)))
    
print("Completed Training!!")



/pytorch/aten/src/ATen/native/BinaryOps.cpp:81: UserWarning: Integer division of tensors using div or / is deprecated, and in a future release div will perform true division as in Python 3. Use true_divide or floor_divide (// in Python) instead.


Epoch [1/10], Iteration [100/600], Training Loss: 0.3547051250934601, training accuracy: 91%
Epoch [1/10], Iteration [200/600], Training Loss: 0.35620513558387756, training accuracy: 92%
Epoch [1/10], Iteration [300/600], Training Loss: 0.19471454620361328, training accuracy: 93%
Epoch [1/10], Iteration [400/600], Training Loss: 0.2647280991077423, training accuracy: 93%
Epoch [1/10], Iteration [500/600], Training Loss: 0.06302858144044876, training accuracy: 94%
Epoch [1/10], Iteration [600/600], Training Loss: 0.17127010226249695, training accuracy: 94%
Epoch [2/10], Iteration [100/600], Training Loss: 0.11000623553991318, training accuracy: 94%
Epoch [2/10], Iteration [200/600], Training Loss: 0.05861934274435043, training accuracy: 95%
Epoch [2/10], Iteration [300/600], Training Loss: 0.13964661955833435, training accuracy: 95%
Epoch [2/10], Iteration [400/600], Training Loss: 0.022027133032679558, training accuracy: 95%
Epoch [2/10], Iteration [500/600], Training Loss: 0.065128758

In [ ]:
# Test network (No weigth updation and weight calculation , no loss calculation)
correct = 0
total = 0
for images,labels in test_loader:
  images = images.view(-1, 28*28)
  if CUDA:
    images = images.cuda()
  outputs = net(images)

  _, predicted = torch.max(outputs.data, 1)
  total += labels.size(0)

  if CUDA:
    correct += (predicted.cpu() == labels.cpu()).sum()
  else:
    correct += (predicted == labels).sum()

print("Final Test Accuracy: %d %%" %(100 * correct / total))


Final Test Accuracy: 98 %
